# 6.0002 Lecture 2: Optimization Problems

**Speaker:** Prof. John Guttag

## The Pros and Cons of Greedy
- easy to implement
- computationally efficient
    - $O(n\log{n})$
- but does not always yield the best solution
    - doesn't solve the **optimization problem**
    - don't even know how good the approximation is

## Brute Force Algorithm
- 1.) enumerate all possible combinations of items
- 2.) remove all of the combinations whose total units exceeds the allowed weight
- 3.) from the remaining combinations, choose any one whose value is largest
    - (in case there are ties)

## Search Tree implementation
- the tree is built top down starting with the root
- the first element is selected from the still to be considered items
    - If there is room for that item in the knapsack, a node is constructed that reflects the consequence of choosing to take that item. By convention, we draw that as the left child
    - We also explore the consequences of not taking that item. This is the right child
- the process is then applied **recursively** to non-leaf children
- finally, chose a node with the highest value that meets constraints

## A search tree enumerates possibilities
- left-first, depth-first enumeration
    - take is left, don't take is right
- once you have all outcomes, compute the value of each to find optimal one

## Computational complexity
- time based on total number of nodes generated
- number of levels is number of items to choose from
- number of nodes at level $i$ is $2^i$
- so, if there are $n$ items, the number of nodes is $\sum_{i=0}^n 2^i$
    - i.e. $O(2^n + 1)$
- an obvious optimization: don't explore parts of tree that violate constraint (e.g. too many calories)
    - doesn't change complexity
- does this mean that brute force is never useful?
    - let's give it a try

## Header for Decision Tree implementation

In [1]:
def maxVal(toConsider, avail):
    """ Assumes toConsider a list of items, 
            avail a weight
        Returns a tuple of the total value of a 
            solution to 0/1 knapsack problem and
            the items of that solution"""

- toConsider: those items that nodes higher up in the tree (corresponding to earlier calls in the recursive call stack) have not yet considered
- avail: the amount of space still available

In [4]:
# code from last time for food objects and menus
class Food(object):
    def __init__(self, n, v, w):
        self.name = n
        self.value = v
        self.calories = w
    def getValue(self):
        return self.value
    def getCost(self):
        return self.calories
    def density(self):
        return self.getValue()/self.getCost()
    def __str__(self):
        return self.name + ': <' + str(self.value)\
                 + ', ' + str(self.calories) + '>'

def buildMenu(names, values, calories):
    menu = []
    for i in range(len(values)):
        menu.append(Food(names[i], values[i],
                          calories[i]))
    return menu

In [8]:
def maxVal(toConsider, avail):
    """ Assumes toConsider a list of items, 
            avail a weight
        Returns a tuple of the total value of a 
            solution to 0/1 knapsack problem and
            the items of that solution"""
    if toConsider == [] or avail == 0:
        result = (0, ())
    elif toConsider[0].getCost() > avail: # (don't take item that goes over bound)
        # explore right branch only
        result = maxVal(toConsider[1:], avail)
    else:
        nextItem = toConsider[0]
        
        # explore left branch
        withVal, withToTake = maxVal(toConsider[1:], avail-nextItem.getCost())
        withVal += nextItem.getValue()
        
        # explore right branch
        withoutVal, withoutToTake = maxVal(toConsider[1:], avail)
        
        # choose better branch
        if withVal > withoutVal:
            result = (withVal, withToTake + (nextItem,))
        else:
            result = (withoutVal, withoutToTake)
    return result

- does not actually build search tree
- local variable *result* records best solution found so far

## Try on example from lecture 1

In [9]:
# test the above implementation
def testMaxVal(foods, maxUnits, printItems = True):
    print('Use search tree to allocate', maxUnits,
          'calories')
    val, taken = maxVal(foods, maxUnits)
    print('Total value of items taken =', val)
    if printItems:
        for item in taken:
            print('   ', item)

names = ['wine', 'beer', 'pizza', 'burger', 'fries',
         'cola', 'apple', 'donut', 'cake']
values = [89,90,95,100,90,79,50,10]
calories = [123,154,258,354,365,150,95,195]
foods = buildMenu(names, values, calories)

testMaxVal(foods, 750)

Use search tree to allocate 750 calories
Total value of items taken = 353
    cola: <79, 150>
    pizza: <95, 258>
    beer: <90, 154>
    wine: <89, 123>


## Search Tree worked great
- gave us a better answer
- finished quickly
- but $2^8$ is not a large number
    - we should look at what happens when we have a more extensive menu from which to choose

## Code to try larger examples

In [11]:
import random

def buildLargeMenu(numItems, maxVal, maxCost):
    items = []
    for i in range(numItems):
        items.append(Food(str(i), 
                         random.randint(1, maxVal), 
                         random.randint(1, maxCost)))
    return items

#for numItems in (5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60):
#    items = buildLargeMenu(numItems, 90, 250)
#    testMaxVal(items, 750, False)

# takes a long time to run...

## Is it hopeless?
- in theory, yes
- in practice, no!
- **Dynamic programming** to the rescue

## The history of Dynamic programming
- Richard Bellman, from the Rand Corporation, chose the name to avoid sounding pejorative 

## Recursive Implementation of Fibonacci

In [13]:
def fib(n):
    if n == 0 or n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)
    
#fib(120)
# will take a long time to run...

## Clearly a bad idea to repeat work
- trade a time for space
- create a table to record what we've done
    - before computing fib(x), check if value of fib(x) already stored in table
        - if so, look it up
        - if not, compute it and then add it to table
    - called **memoization**
        - i.e. create a memo, and store it in the memo

## Using a memo to compute Fibonacci

In [14]:
def fastFib(n, memo={}):
    """ Assumes n is an int >= 0, memo used only by recursive calls
        Returns Fibonacci of n"""
    if n == 0 or n == 1:
        return 1
    try:
        return memo[n]
    except KeyError: # not to handle errors, but just as flow control
        result = fastFib(n-1, memo) + fastFib(n-2, memo)
        memo[n] = result
        return result

In [15]:
fastFib(120) # runs very quickly!

8670007398507948658051921

## When does it work?
- **Optimal substructure:** a globally optimal solution can be found by combining optimal solutions to local subproblems
    - for x > 1, fib(x) = fib(x-1) + fib(x-2)
- **Overlapping subproblems:** finding an optimal solution involves solving the same problem multiple times
    - compute fib(x) many times

## What about 0/1 Knapsack problem?
- do these conditions hold?
    - optimal substructure?
        - yes: choosing winner between left branch and right branch
    - overlapping subproblems?
        - no: none of the nodes are identical...?
        - could work if some of the items are identical

## What problem is solved at each node?
- given remaining weight, maximize value by choosing among remaining items
- set of previously chosen items, or even value of that set, doesn't matter!
    - so we actually can have overlapping subproblems

## Modify maxVal to use a Memo
- add memo as a third argument
    - def fastMaxVal(toConsider, avail, memo={})
- key of memo is tuple
    - (items left to be considered, available weight)
    - items left to be considered represented by len(toConsider)
- first thing body of function does is check whether the optimal choice of items given the available weight is already in the memo
- last thing body of function does is update the memo

## How can the performance be so good?
- problem is inherently exponential
- have we overturned the laws of the universe?
- is dynamic programming a miracle?
- no, but computational complexity can be subtle
- running time of fastMaxVal is governed by the number of distinct pairs, <toConsider, avail>
    - number of possible values of toConsider bounded by len(items)
    - possible values of avail a bit harder to charactierize
        - bounded by number of distinct sums of weights
    - covered in more detail in assigned reading

## Summary of lectures 1-2
- many problems of practical importance can be formulated as **optimization problems**
- **greedy algorithms** often provide adequate (though not necessarily optimal) solutions
- finding an optimal solution is usually **exponentially hard**
- but **dynamic programing** often yields good performance for a subclass of optimization problems-- those with optimal substructure and overlapping subproblems
    - solutions always correct
    - fast under the right circumstances